In [9]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI 
import google.generativeai
import anthropic

import gradio as gr 

In [10]:
# Load Environment Variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
#anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY is not set")
#if anthropic_api_key is None:
#    raise ValueError("ANTHROPIC_API_KEY is not set")
#if google_api_key is None:
#    raise ValueError("GOOGLE_API_KEY is not set")

# Connect to OpenAI
openai = OpenAI()

# Connect to Anthropic
#claude = anthropic.Anthropic()

# Connect to Google
#google = google.generativeai.GenerativeModel()

# Asking LLMs to ask a question

We want to ask this question: outline a business plan for a new AI-powered chatbot.

### What information is included in the API

Typically we'll pass to the API:

1- The name of the model that should be used \
2- A system message that gives overall context for the role the LLM is playing\
3- A user message that provides the actual prompt\

There are other parameters that can be used, including *temperature* which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [12]:
# A generic system message
system_message = """
You are a helpful assistant that can answer questions and help with tasks.
"""

# A generic user message
user_message = "What is the capital of the moon?"

In [15]:
# A general function for the chat interface
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    # Get the response from the model
    response = openai.chat.completions.create(
        model="chatgpt-4o-latest",
        messages=messages
    )
    return response.choices[0].message.content

In [16]:
message_gpt(user_message)

'The Moon does not have a capital, as it is not governed by any country or entity. However, in science fiction and speculative discussions, various fictional lunar cities or colonies have been imagined, such as *Artemis* (from Andy Weir’s novel *Artemis*) or *Moonbase Alpha* (from the TV series *Space: 1999*). \n\nIf humans establish a permanent settlement on the Moon in the future, perhaps a "capital" may eventually be designated! 🚀🌕'

# User Interface Gradio

In [24]:
# User Interface
with gr.Blocks() as demo:
    gr.Markdown("Hello World")
demo.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


In [25]:
# Simple Chat Interface
def shout(message):
    return message.upper()

demo = gr.Interface(fn=shout, inputs="text", outputs="text")
demo.launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [26]:
# Remove flags
# Adding share=True means that it can be accessed publically
gr.Interface(fn=shout, inputs="text", outputs="text", flagging_mode="never").launch()


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [27]:
# Adding inbrowser=True opens up a new browser window automatically
gr.Interface(fn=shout, inputs="text", outputs="text", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


# Forcing dark mode

In [64]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


# Add more inputs and outputs

In [30]:
view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Enter a message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
    description="Enter a message and get the uppercase version",
)
view.launch()


* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


In [47]:
# Now change "shout" to " message_gpt"

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Enter a message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


In [67]:
# Let's create a call that streams back results
def stream_message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [68]:
view = gr.Interface(
    fn=stream_message_gpt,
    inputs=[gr.Textbox(label="Enter a message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
    
)
view.launch()

* Running on local URL:  http://127.0.0.1:7899

To create a public link, set `share=True` in `launch()`.


In [36]:
# for other models
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [69]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_message_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [70]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


# Building a company brochure generator

In [75]:
# A class to represent a Webpage

class Webpage:
    url: str
    title: str
    text: str
    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
  

In [76]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

def stream_brochure(company_name, url, model):
    prompt = f"Company Name: {company_name}\nCompany Website: {url}\n{Webpage(url).get_contents()}"
    if model=="GPT":
        result = stream_message_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [77]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[gr.Textbox(label="Company Name:"), 
            gr.Textbox(label="Company Website:"), 
            gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7902

To create a public link, set `share=True` in `launch()`.
